In [1]:
!pip install transformers datasets torch pandas openpyxl
!pip install transformers torch deep-translator langdetect
!pip install streamlit
!pip install pyngrok
import os
import torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

In [ ]:
!huggingface-cli login

In [2]:
import pandas as pd
df = pd.read_excel("/content/drive/MyDrive/guvidataset.xlsx")
print(df.head())

                                        Question  \
0                                  What is GUVI?   
1                 What is the main goal of GUVI?   
2            What domains do GUVI courses cover?   
3  What are the types of learning modes in GUVI?   
4                  What are GUVI's free courses?   

                                              Answer  
0  GUVI is an Indian ed-tech platform started in ...  
1  To democratize tech education for non-English-...  
2  Programming, Web Development, AI/ML, Cybersecu...  
3  Self-paced courses, Zen Class live programs, a...  
4  Python, Java, ChatGPT & AI, JavaScript for Beg...  


In [3]:
df["text"] = "Q: " + df["Question"].astype(str) + "\nA: " + df["Answer"].astype(str)
print(df["text"].head())

0    Q: What is GUVI?\nA: GUVI is an Indian ed-tech...
1    Q: What is the main goal of GUVI?\nA: To democ...
2    Q: What domains do GUVI courses cover?\nA: Pro...
3    Q: What are the types of learning modes in GUV...
4    Q: What are GUVI's free courses?\nA: Python, J...
Name: text, dtype: object


In [4]:
from datasets import Dataset
dataset = Dataset.from_pandas(df[["text"]])
print(dataset)

Dataset({
    features: ['text'],
    num_rows: 561
})


In [5]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("gpt2")
# Add pad token if missing
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def tokenize(batch):
    tokens = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)
    tokens["labels"] = tokens["input_ids"].copy()   # ✅ important for loss calculation
    return tokens

tokenized_ds = dataset.map(tokenize, batched=True, remove_columns=["text"])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Map:   0%|          | 0/561 [00:00<?, ? examples/s]

In [6]:
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained("gpt2")
model.resize_token_embeddings(len(tokenizer))

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Embedding(50257, 768)

In [7]:
os.environ["WANDB_MODE"] = "disabled"

In [8]:
# Split dataset
dataset_split = dataset.train_test_split(test_size=0.1)
# Tokenize with labels
def tokenize(batch):
    tokens = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_split = dataset_split.map(tokenize, batched=True, remove_columns=["text"])

Map:   0%|          | 0/504 [00:00<?, ? examples/s]

Map:   0%|          | 0/57 [00:00<?, ? examples/s]

In [9]:
print("CUDA available:", torch.cuda.is_available())
print("Device:", torch.device("cuda" if torch.cuda.is_available() else "cpu"))

CUDA available: False
Device: cpu


In [10]:
import torch
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./guvi-gpt2",
    overwrite_output_dir=True,
    do_eval=True,
    learning_rate=5e-5,
    per_device_train_batch_size=2,
    num_train_epochs=1,      # one pass through dataset
    max_steps=200,           # stop early after 200 steps
    weight_decay=0.01,
    save_total_limit=1,
    save_strategy="steps",
    save_steps=100,
    fp16=torch.cuda.is_available(),
    logging_dir="./logs",
    logging_steps=20,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_split["train"],
    eval_dataset=tokenized_split["test"],
    tokenizer=tokenizer,
)

/tmp/ipython-input-4172515846.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [11]:
trainer.train()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
20,1.350500
40,0.354200
60,0.294400
80,0.304100
100,0.275500
120,0.277600
140,0.269400
160,0.302400
180,0.281500
200,0.273300


TrainOutput(global_step=200, training_loss=0.39828550338745117, metrics={'train_runtime': 1970.3579, 'train_samples_per_second': 0.203, 'train_steps_per_second': 0.102, 'total_flos': 52258406400000.0, 'train_loss': 0.39828550338745117, 'epoch': 0.7936507936507936})

In [12]:
trainer.save_model("./guvi-gpt2-model")
tokenizer.save_pretrained("./guvi-gpt2-model")


('./guvi-gpt2-model/tokenizer_config.json',
 './guvi-gpt2-model/special_tokens_map.json',
 './guvi-gpt2-model/vocab.json',
 './guvi-gpt2-model/merges.txt',
 './guvi-gpt2-model/added_tokens.json',
 './guvi-gpt2-model/tokenizer.json')

In [13]:
from deep_translator import GoogleTranslator
from langdetect import detect
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# =========================
# 1. Translation Functions
# =========================
def to_english(text):
    lang = detect(text)
    if lang == "en":
        return text, "en"
    translated = GoogleTranslator(source=lang, target="en").translate(text)
    return translated, lang

def from_english(text, target_lang):
    if target_lang == "en":
        return text
    return GoogleTranslator(source="en", target=target_lang).translate(text)

# =========================
# 2. Load GPT-2 Model
# =========================
model_path = "/content/guvi-gpt2-model"   # change if needed
tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# =========================
# 3. Text Generation
# =========================
def generate_response(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# =========================
# 4. Multilingual Chat
# =========================
def multilingual_chat(user_input):
    # Translate user input → English
    eng_input, lang = to_english(user_input)

    # Generate GPT-2 response (English)
    eng_response = generate_response(eng_input)

    # Translate back → Original Language
    final_response = from_english(eng_response, lang)

    return final_response

# =========================
# 5. Test
# =========================
print(multilingual_chat("வணக்கம், நீங்கள் என்ன செய்கிறீர்கள்?"))  # Tamil
print(multilingual_chat("Hola, ¿qué tal?"))                     # Spanish
print(multilingual_chat("Hello, how are you?"))                 # English


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


வணக்கம், நீங்கள் என்ன செய்கிறீர்கள்? உங்கள் பெயர் என்ன? உங்கள் நிறுவனம் என்ன?
ப: ஊடாடும் தரவு அறிவியல், பகுப்பாய்வு மற்றும் இயந்திர கற்றல் ஆகியவற்றின் லட்டுகள்.


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hola, què passa amb?
R: Podeu sol·licitar assistència al desenvolupador mitjançant correu electrònic, xat en directe i fòrums.
Hello, how are you? 
A: I'm a PhD candidate at the VBA Cybersecurity Division.


In [14]:
%%writefile app.py
import streamlit as st
from deep_translator import GoogleTranslator
from langdetect import detect
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch

# =========================
# Translation Functions
# =========================
def to_english(text):
    lang = detect(text)
    if lang == "en":
        return text, "en"
    translated = GoogleTranslator(source=lang, target="en").translate(text)
    return translated, lang

def from_english(text, target_lang):
    if target_lang == "en":
        return text
    return GoogleTranslator(source="en", target=target_lang).translate(text)

# =========================
# Load GPT-2 Model
# =========================
@st.cache_resource
def load_model():
    model_path = "./guvi-gpt2-model"
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2LMHeadModel.from_pretrained(model_path)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()
    return tokenizer, model, device

tokenizer, model, device = load_model()

# =========================
# Text Generation
# =========================
def generate_response(prompt, max_new_tokens=100):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=0.8
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# =========================
# Multilingual Chat Function
# =========================
def multilingual_chat(user_input):
    eng_input, lang = to_english(user_input)
    eng_response = generate_response(eng_input)
    final_response = from_english(eng_response, lang)
    return final_response

# =========================
# Streamlit UI
# =========================
st.set_page_config(page_title="🌐 GUVI Multilingual Chatbot", layout="wide")

# CSS Styling
st.markdown("""
    <style>
        .main {
            background-color: #f4f6f7;
            padding: 2rem;
        }
        h1, h3 {
            font-family: 'Arial Black';
            color: #2c3e50;
        }
        .stTextInput > div > div > input {
            font-size: 18px;
            padding: 10px;
        }
        .response-box {
            background-color: #dff9fb;
            border-radius: 10px;
            padding: 1rem;
            margin-top: 1rem;
        }
        .footer {
            text-align: center;
            font-size: 14px;
            color: gray;
        }
    </style>
""", unsafe_allow_html=True)

# =========================
# Header
# =========================
col1, col2, col3 = st.columns([1, 3, 1])
with col2:
    st.image("https://tse2.mm.bing.net/th/id/OIP.pKsMMQkzvS9RrCz8HYtR5AHaD4?r=0&rs=1&pid=ImgDetMain&o=7&rm=3", width=150)

st.markdown("<h1 style='text-align: center;'>🌐 GUVI Multilingual GPT-2 Chatbot</h1>", unsafe_allow_html=True)
st.markdown("<p style='text-align: center;'>Talk to me in any language – I'll translate, generate, and reply back smartly! 🚀</p>", unsafe_allow_html=True)
st.markdown("---")

# =========================
# Chat Interface
# =========================
st.subheader("💬 Ask Me Anything")
user_input = st.text_input("Type your question here...", key="input")

if user_input:
    with st.spinner("🤖 Thinking and Translating..."):
        response = multilingual_chat(user_input)
    st.markdown(f"<div class='response-box'><strong>🤖 Bot:</strong> {response}</div>", unsafe_allow_html=True)

# =========================
# Footer
# =========================
st.markdown("---")
st.markdown("<div class='footer'>🚀 Powered by Ranjitha | Built with ❤️ using Streamlit & Hugging Face</div>", unsafe_allow_html=True)


Writing app.py


In [ ]:
!npm install localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇
added 22 packages in 2s
⠇
⠇3 packages are looking for funding
⠇  run `npm fund` for details
⠇

In [ ]:
!streamlit run /content/app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

35.199.12.155
⠙your url is: https://great-paths-tap.loca.lt
/content/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:10425 (check your firewall settings)
    at Socket.<anonymous> (/content/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0
⠙

In [23]:
trainer.save_model("./guvi-gpt2-model")
tokenizer.save_pretrained("./guvi-gpt2-model")


('./guvi-gpt2-model/tokenizer_config.json',
 './guvi-gpt2-model/special_tokens_map.json',
 './guvi-gpt2-model/vocab.json',
 './guvi-gpt2-model/merges.txt',
 './guvi-gpt2-model/added_tokens.json')

In [24]:
# Run in terminal or a Colab cell
!ls -la ./guvi-gpt2-model


total 491044
drwxr-xr-x 2 root root      4096 Aug  9 07:03 .
drwxr-xr-x 1 root root      4096 Aug  9 07:03 ..
-rw-r--r-- 1 root root       880 Aug  9 07:27 config.json
-rw-r--r-- 1 root root       119 Aug  9 07:27 generation_config.json
-rw-r--r-- 1 root root    456318 Aug  9 07:27 merges.txt
-rw-r--r-- 1 root root 497774208 Aug  9 07:27 model.safetensors
-rw-r--r-- 1 root root       583 Aug  9 07:27 special_tokens_map.json
-rw-r--r-- 1 root root       556 Aug  9 07:27 tokenizer_config.json
-rw-r--r-- 1 root root   3557957 Aug  9 07:27 tokenizer.json
-rw-r--r-- 1 root root      5304 Aug  9 07:27 training_args.bin
-rw-r--r-- 1 root root    999186 Aug  9 07:27 vocab.json


In [ ]:
from huggingface_hub import create_repo, upload_folder

# 🔑 Your Hugging Face token (with Write access)
hf_token = "hf_token"

# ⚡ Repo under your username
repo_id = "ranji30798/chatbot"

# 1. Create repo (safe even if it already exists)
create_repo(repo_id, private=False, exist_ok=True, token=hf_token)

# 2. Upload model folder (make sure it exists in Colab)
upload_folder(
    repo_id=repo_id,
    folder_path="/content/guvi-gpt2-model",   # trainer.save_model() saved here
    commit_message="Upload GUVI fine-tuned GPT-2 model",
    token=hf_token
)

print(f"✅ Model uploaded! Check: https://huggingface.co/{repo_id}")

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...t/guvi-gpt2-model/training_args.bin: 100%|##########| 5.30kB / 5.30kB            

  ...t/guvi-gpt2-model/model.safetensors:   5%|5         | 25.1MB /  498MB            

No files have been modified since last commit. Skipping to prevent empty commit.


✅ Model uploaded! Check: https://huggingface.co/ranji30798/chatbot


In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
from huggingface_hub import login

# Login with your token (only once, or set HF_TOKEN environment variable)
login(token="hf_token")

model_path = "ranji30798/chatbot"

tokenizer = GPT2Tokenizer.from_pretrained(model_path, use_auth_token=True)
model = GPT2LMHeadModel.from_pretrained(model_path, use_auth_token=True)


In [35]:
model_path = "./guvi-gpt2-model"

tokenizer = GPT2Tokenizer.from_pretrained(model_path)
model = GPT2LMHeadModel.from_pretrained(model_path)


In [36]:
from google.colab import drive
drive.mount('/content/drive')

# Then use the path where your model is stored, e.g.:
model_path = "/content/drive/MyDrive/guvi-gpt2-model"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from huggingface_hub import login
login(token="hf_token")


In [39]:
model_path = "ranji30798/chatbot"

tokenizer = GPT2Tokenizer.from_pretrained(model_path, use_auth_token=True)
model = GPT2LMHeadModel.from_pretrained(model_path, use_auth_token=True)


In [40]:
!ls guvi-gpt2-model


config.json		model.safetensors	 tokenizer.json
generation_config.json	special_tokens_map.json  training_args.bin
merges.txt		tokenizer_config.json	 vocab.json


In [41]:
!ls ./guvi-gpt2-model

config.json		model.safetensors	 tokenizer.json
generation_config.json	special_tokens_map.json  training_args.bin
merges.txt		tokenizer_config.json	 vocab.json
